# 30+ Essential Pandas Methods Tutorial

A comprehensive guide to the most useful pandas methods using a Yamaha motorcycles dataset.

## 1. Setup and Data Loading

In [ ]:
# First, update your setup cell
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings

# Configure pandas to not use inf as null
pd.options.mode.use_inf_as_na = False

# Suppress warnings
warnings.filterwarnings('ignore')

# Set up plotting configurations
%matplotlib inline
plt.style.use('seaborn-v0_8')  # Use updated style name
plt.rcParams['figure.figsize'] = [12, 8]

In [ ]:
# 1. Read Excel file
df = pd.read_excel('yamaha_motrocycles.xlsx')

# 2. Create a copy for safe manipulation
df_clean = df.copy()

print('Data loaded successfully!')
df.head()

## 2. Basic Data Inspection Methods

In [ ]:
# 3. Basic information about the dataset
print('Dataset Info:')
df.info()

# 4. Get column names
print('\nColumns:', df.columns.tolist())

# 5. Get data types
print('\nData Types:')
print(df.dtypes)

# 6. Check for missing values
print('\nMissing Values:')
print(df.isnull().sum())

# 7. Basic statistics
print('\nBasic Statistics:')
display(df.describe())

## 3. Data Selection and Filtering Methods

In [ ]:
# 8. Select specific columns
if 'Price' in df.columns and 'Model' in df.columns:
    selected_cols = df[['Model', 'Price']]
    print('Selected columns:')
    display(selected_cols.head())

# 9. Filter rows by condition
if 'Price' in df.columns:
    expensive_bikes = df[df['Price'] > df['Price'].mean()]
    print('\nExpensive bikes:')
    display(expensive_bikes.head())

# 10. Multiple conditions
if all(col in df.columns for col in ['Price', 'Year']):
    filtered_df = df[(df['Price'] > 10000) & (df['Year'] > 2020)]
    print('\nNew expensive bikes:')
    display(filtered_df.head())

# 11. Select by index
print('\nFirst 3 rows using iloc:')
display(df.iloc[0:3])

# 12. Select by label
if 'Model' in df.columns:
    print('\nSelect by label using loc:')
    display(df.loc[df['Model'].str.contains('R1', na=False)])

## 4. Data Cleaning Methods

In [ ]:
# 13. Remove duplicates
df_clean = df_clean.drop_duplicates()

# 14. Handle missing values
numeric_cols = df_clean.select_dtypes(include=['number']).columns
categorical_cols = df_clean.select_dtypes(include=['object']).columns

# 15. Fill numeric missing values with mean
df_clean[numeric_cols] = df_clean[numeric_cols].fillna(df_clean[numeric_cols].mean())

# 16. Fill categorical missing values with mode
df_clean[categorical_cols] = df_clean[categorical_cols].fillna(df_clean[categorical_cols].mode().iloc[0])

# 17. Remove outliers (example for Price)
if 'Price' in df_clean.columns:
    Q1 = df_clean['Price'].quantile(0.25)
    Q3 = df_clean['Price'].quantile(0.75)
    IQR = Q3 - Q1
    df_clean = df_clean[(df_clean['Price'] >= Q1 - 1.5*IQR) & 
                        (df_clean['Price'] <= Q3 + 1.5*IQR)]

print('Data cleaning completed!')

## 5. Data Transformation Methods

In [ ]:
# 18. Create new calculated column
if all(col in df_clean.columns for col in ['Price', 'Year']):
    df_clean['Age'] = 2024 - df_clean['Year']
    df_clean['Price_per_Year'] = df_clean['Price'] / df_clean['Age']

# 19. Apply custom function
if 'Price' in df_clean.columns:
    df_clean['Price_Category'] = df_clean['Price'].apply(
        lambda x: 'High' if x > df_clean['Price'].mean() else 'Low')

# 20. String operations
if 'Model' in df_clean.columns:
    df_clean['Model_Upper'] = df_clean['Model'].str.upper()

# 21. Binning
if 'Price' in df_clean.columns:
    df_clean['Price_Bins'] = pd.qcut(df_clean['Price'], q=4, labels=['Budget', 'Mid', 'Premium', 'Luxury'])

# 22. One-hot encoding
if len(categorical_cols) > 0:
    df_clean = pd.get_dummies(df_clean, columns=[categorical_cols[0]], prefix='cat')

print('Transformations completed!')
display(df_clean.head())

## 6. Aggregation and Grouping Methods

In [ ]:
# 23. Basic groupby operations
if 'Price_Category' in df_clean.columns:
    group_stats = df_clean.groupby('Price_Category').agg({
        'Price': ['count', 'mean', 'median', 'std'],
        'Age': 'mean'
    })
    print('Group statistics:')
    display(group_stats)

# 24. Pivot table
if all(col in df_clean.columns for col in ['Year', 'Price_Category', 'Price']):
    pivot_table = pd.pivot_table(
        df_clean,
        values='Price',
        index='Year',
        columns='Price_Category',
        aggfunc='mean'
    )
    print('\nPivot table:')
    display(pivot_table)

# 25. Rolling calculations
if 'Price' in df_clean.columns:
    df_clean['Rolling_Avg_Price'] = df_clean['Price'].rolling(window=3).mean()

# 26. Cumulative calculations
if 'Price' in df_clean.columns:
    df_clean['Cumulative_Sum'] = df_clean['Price'].cumsum()

## 7. Advanced Analysis Methods

In [ ]:
# 27. Correlation analysis
numeric_correlation = df_clean.select_dtypes(include=['number']).corr()
print('Correlation matrix:')
display(numeric_correlation)

# 28. Value counts with percentages
if 'Price_Category' in df_clean.columns:
    value_counts = df_clean['Price_Category'].value_counts(normalize=True) * 100
    print('\nPrice category distribution (%):')
    display(value_counts)

# 29. Descriptive statistics by category
if 'Price_Category' in df_clean.columns:
    category_stats = df_clean.groupby('Price_Category').describe()
    print('\nDetailed statistics by category:')
    display(category_stats)

# 30. Rank items
if 'Price' in df_clean.columns:
    df_clean['Price_Rank'] = df_clean['Price'].rank(method='dense', ascending=False)

## 8. Visualization Methods

In [ ]:
# Create visualizations with error handling
try:
    # Create figure and axes
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 15))

    # 31. Histogram
    if 'Price' in df_clean.columns:
        ax1.hist(df_clean['Price'].dropna(), bins=30)
        ax1.set_title('Price Distribution')
        ax1.set_xlabel('Price')
        ax1.set_ylabel('Count')

    # 32. Box plot
    if 'Price_Category' in df_clean.columns and 'Price' in df_clean.columns:
        sns.boxplot(x='Price_Category', y='Price', data=df_clean.dropna(), ax=ax2)
        ax2.set_title('Price by Category')

    # 33. Scatter plot
    if all(col in df_clean.columns for col in ['Age', 'Price']):
        sns.scatterplot(x='Age', y='Price', data=df_clean.dropna(), ax=ax3)
        ax3.set_title('Price vs Age')

    # 34. Correlation heatmap
    corr_data = df_clean.select_dtypes(include=[np.number]).corr()
    sns.heatmap(corr_data, 
                annot=True,
                fmt='.2f',
                cmap='coolwarm',
                center=0,
                ax=ax4)
    ax4.set_title('Correlation Heatmap')

    # Adjust layout and display
    plt.tight_layout()
    plt.show()

except Exception as e:
    print(f"Error in visualization: {str(e)}")
    plt.close()

## 9. Export and Save Results

In [ ]:
# 35. Export to Excel with multiple sheets
with pd.ExcelWriter('analysis_results.xlsx') as writer:
    df_clean.to_excel(writer, sheet_name='Clean_Data', index=False)
    numeric_correlation.to_excel(writer, sheet_name='Correlations')
    if 'Price_Category' in df_clean.columns:
        group_stats.to_excel(writer, sheet_name='Group_Statistics')

print('Analysis results saved to analysis_results.xlsx')

## Summary

This notebook demonstrated 35 essential pandas methods across different categories:

1. **Basic Operations**: read_excel, info, head, describe
2. **Selection**: iloc, loc, boolean indexing
3. **Cleaning**: drop_duplicates, fillna, handling outliers
4. **Transformation**: apply, string operations, binning
5. **Aggregation**: groupby, pivot_table, rolling calculations
6. **Analysis**: correlation, value_counts, rank
7. **Visualization**: histplot, boxplot, heatmap
8. **Export**: to_excel with multiple sheets

Each method includes error handling and conditional checks to ensure the code runs with different dataset structures.